<a href="https://colab.research.google.com/github/soyyoon/Sign2Talk/blob/main/classification_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q scikit-learn seaborn

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import glob
from tqdm import tqdm
from scipy.ndimage import gaussian_filter1d
import xgboost as xgb
from google.colab import drive
drive.mount('/content/drive')

# Seed
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

DATA_DIR = '/content/drive/MyDrive/S2T/features_basic_47dim'
MAX_FRAMES = 30
FEATURE_DIM = 47
VAL_SPLIT_SIZE = 0.2
BATCH_SIZE = 32

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 2. 데이터 로딩 및 TF Dataset 헬퍼

In [ ]:
def load_all_data(data_dir, max_frames=30, feature_dim=47):
    """NPY 파일 로드"""
    print("="*60)
    print("📦 데이터 로딩")
    print("="*60)

    X_data = []
    y_data = []

    class_names = sorted([d for d in os.listdir(data_dir)
                          if os.path.isdir(os.path.join(data_dir, d))
                          and not d.startswith('.')])

    print(f"📂 클래스: {len(class_names)}개")

    for class_idx, class_name in enumerate(tqdm(class_names, desc="Loading")):
        class_path = os.path.join(data_dir, class_name)
        npy_files = glob.glob(os.path.join(class_path, "*.npy"))

        for fpath in npy_files:
            try:
                arr = np.load(fpath)
                if arr.shape == (max_frames, feature_dim):
                    X_data.append(arr)
                    y_data.append(class_idx)
            except Exception as e:
                print(f"Warning: Skipping file {fpath} due to error: {e}")
                continue

    X_data = np.array(X_data, dtype=np.float32)
    y_data = np.array(y_data, dtype=np.int32)

    print(f"\n✅ 총 샘플: {len(X_data)}개")
    return X_data, y_data, class_names


def create_augmented_dataset(X, y, batch_size=32, augment_fn=None):
    """
    Numpy 배열과 증강 함수를 받아 tf.data.Dataset 생성
    """
    dataset = tf.data.Dataset.from_tensor_slices((X, y))
    dataset = dataset.shuffle(len(X), seed=SEED)

    if augment_fn:
        def tf_augment_fn(x, y):
            # tf.py_function을 사용하여 Numpy 기반 증강 함수 래핑
            [x_aug] = tf.py_function(
                func=lambda b: [augment_fn(b.numpy()[np.newaxis, ...])[0]],
                inp=[x],
                Tout=[tf.float32]
            )
            x_aug.set_shape((MAX_FRAMES, FEATURE_DIM))
            return x_aug, y

        # 증강 적용
        dataset = dataset.map(tf_augment_fn, num_parallel_calls=tf.data.AUTOTUNE)

    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

## 3. 손실 함수 (Label Smoothing)

In [ ]:
def smooth_sparse_categorical_crossentropy(y_true, y_pred, label_smoothing=0.1):
    num_classes = tf.shape(y_pred)[-1]
    y_true = tf.cast(y_true, tf.int32)
    y_true_one_hot = tf.one_hot(y_true, depth=num_classes)

    y_true_smoothed = y_true_one_hot * (1 - label_smoothing) + \
                      (label_smoothing / tf.cast(num_classes, tf.float32))

    loss = -tf.reduce_sum(
        y_true_smoothed * tf.math.log(tf.clip_by_value(y_pred, 1e-7, 1.0)),
        axis=-1
    )
    return tf.reduce_mean(loss)

## 4. 모델 아키텍쳐

In [ ]:
def build_cnn_attention_deep(input_shape, num_classes):
    """더 깊은 CNN + Attention"""
    inp = layers.Input(shape=input_shape)
    x = layers.Conv1D(128, 3, padding="same")(inp)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Conv1D(128, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Conv1D(256, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Conv1D(256, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Dropout(0.2)(x)
    attn_scores = layers.Dense(1)(x)
    attn_scores = layers.Softmax(axis=1)(attn_scores)
    def weighted_sum(args):
        feats, scores = args
        return tf.reduce_sum(feats * scores, axis=1)
    x = layers.Lambda(weighted_sum)([x, attn_scores])
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(256, activation="relu")(x)
    x = layers.Dropout(0.4)(x)
    x = layers.Dense(128, activation="relu")(x)
    x = layers.Dropout(0.3)(x)
    out = layers.Dense(num_classes, activation="softmax")(x)
    return models.Model(inp, out)


def build_bilstm_attention(input_shape, num_classes):
    """BiLSTM + Multi-head Attention"""
    inp = layers.Input(shape=input_shape)
    x = layers.Conv1D(128, 3, padding="same")(inp)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(x)
    x = layers.Dropout(0.3)(x)
    attn_output = layers.MultiHeadAttention(
        num_heads=4, key_dim=64, dropout=0.2
    )(x, x)
    x = layers.LayerNormalization()(x + attn_output)
    avg_pool = layers.GlobalAveragePooling1D()(x)
    max_pool = layers.GlobalMaxPooling1D()(x)
    x = layers.Concatenate()([avg_pool, max_pool])
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    out = layers.Dense(num_classes, activation='softmax')(x)
    return models.Model(inp, out)


def build_transformer_model(input_shape, num_classes):
    """Transformer Encoder"""
    inp = layers.Input(shape=input_shape)
    x = layers.Dense(128)(inp)
    x = layers.LayerNormalization()(x)
    positions = tf.range(start=0, limit=input_shape[0], delta=1)
    position_embedding = layers.Embedding(
        input_dim=input_shape[0], output_dim=128
    )(positions)
    x = x + position_embedding
    for _ in range(3):
        attn_output = layers.MultiHeadAttention(
            num_heads=4, key_dim=32, dropout=0.2
        )(x, x)
        x = layers.LayerNormalization()(x + attn_output)
        ffn = tf.keras.Sequential([
            layers.Dense(256, activation="relu"),
            layers.Dropout(0.2),
            layers.Dense(128),
        ])
        ffn_output = ffn(x)
        x = layers.LayerNormalization()(x + ffn_output)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(128, activation="relu")(x)
    x = layers.Dropout(0.4)(x)
    out = layers.Dense(num_classes, activation="softmax")(x)
    return models.Model(inp, out)


def build_resnet_1d(input_shape, num_classes):
    """1D ResNet"""
    inp = layers.Input(shape=input_shape)
    def residual_block(x, filters, kernel_size=3):
        shortcut = x
        x = layers.Conv1D(filters, kernel_size, padding="same")(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation('relu')(x)
        x = layers.Dropout(0.2)(x)
        x = layers.Conv1D(filters, kernel_size, padding="same")(x)
        x = layers.BatchNormalization()(x)
        if shortcut.shape[-1] != filters:
            shortcut = layers.Conv1D(filters, 1, padding="same")(shortcut)
        x = layers.Add()([x, shortcut])
        x = layers.Activation('relu')(x)
        return x
    x = layers.Conv1D(64, 7, padding="same")(inp)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = residual_block(x, 64)
    x = residual_block(x, 128)
    x = residual_block(x, 256)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(128, activation="relu")(x)
    x = layers.Dropout(0.5)(x)
    out = layers.Dense(num_classes, activation="softmax")(x)
    return models.Model(inp, out)

## 5. 데이터 증강

In [ ]:
def mega_augment_batch(batch):
    """최강 증강 (배치 단위가 아닌 단일 샘플 [1, T, F] 대상)"""
    # 입력이 (1, T, F)이므로 [0]을 통해 (T, F)로 작업
    augmented = batch[0].copy()

    if np.random.rand() < 0.8:
        noise_level = np.random.uniform(0.01, 0.02)
        noise = np.random.normal(0, noise_level, size=augmented.shape)
        augmented = augmented + noise
    if np.random.rand() < 0.6:
        shift = np.random.choice([-3, -2, -1, 1, 2, 3])
        augmented = np.roll(augmented, shift, axis=0) # axis=0 (시간)
    if np.random.rand() < 0.5:
        scale = np.random.uniform(0.85, 1.15)
        augmented = augmented * scale
    if np.random.rand() < 0.4:
        mask_len = np.random.randint(1, 5)
        mask_start = np.random.randint(0, MAX_FRAMES - mask_len)
        augmented[mask_start:mask_start+mask_len, :] = 0
    if np.random.rand() < 0.4:
        mask_features = np.random.randint(1, 8)
        feature_indices = np.random.choice(FEATURE_DIM, mask_features, replace=False)
        augmented[:, feature_indices] = 0
    if np.random.rand() < 0.2:
        augmented = augmented[::-1, :] # axis=0 (시간)
    if np.random.rand() < 0.3:
        sigma = np.random.uniform(0.5, 1.5)
        augmented = gaussian_filter1d(augmented, sigma=sigma, axis=0) # axis=0 (시간)

    # (T, F) -> (1, T, F)로 복원
    return augmented.astype(np.float32)[np.newaxis, ...]

## 6. 하이브리드 앙상블 학습

In [ ]:
def train_hybrid_ensemble(X_train_norm, X_val_norm, y_train, y_val,
                          INPUT_SHAPE, NUM_CLASSES,
                          train_ds, val_ds, class_weights):
    """다양한 구조의 모델을 앙상블 (정규화된 데이터 입력)"""

    model_builders = [
        ('CNN_Attention_v1', build_cnn_attention_deep, 0.001),
        ('CNN_Attention_v2', build_cnn_attention_deep, 0.0005),
        ('BiLSTM_Attention_v1', build_bilstm_attention, 0.001),
        ('BiLSTM_Attention_v2', build_bilstm_attention, 0.0005),
        ('Transformer_v1', build_transformer_model, 0.0008),
        ('ResNet1D_v1', build_resnet_1d, 0.001),
        ('ResNet1D_v2', build_resnet_1d, 0.0005),
    ]

    base_models = []
    model_names = []

    for name, builder, lr in model_builders:
        print(f"\n{'='*60}")
        print(f"🏗️ {name} 학습 중... (LR: {lr})")
        print(f"{'='*60}")

        tf.random.set_seed(SEED + len(base_models)) # 모델별 시드 변경
        model = builder(INPUT_SHAPE, NUM_CLASSES)
        optimizer = tf.keras.optimizers.Adam(learning_rate=lr, clipnorm=1.0)

        model.compile(
            optimizer=optimizer,
            loss=smooth_sparse_categorical_crossentropy,
            metrics=['accuracy']
        )

        model.fit(
            train_ds,
            validation_data=val_ds,
            epochs=100, # EarlyStopping이 관리
            callbacks=[
                callbacks.EarlyStopping(
                    monitor='val_loss', patience=15, restore_best_weights=True, verbose=1
                ),
                callbacks.ReduceLROnPlateau(
                    monitor='val_loss', factor=0.5, patience=5, min_lr=1e-7, verbose=1
                )
            ],
            class_weight=class_weights,
            verbose=1 # 학습 과정 모니터링
        )

        # 검증셋 평가 (정규화된 X_val_norm 사용)
        val_preds = model.predict(X_val_norm, batch_size=BATCH_SIZE, verbose=0)
        val_acc = accuracy_score(y_val, np.argmax(val_preds, axis=1))
        print(f"  ✅ {name} Val Acc: {val_acc*100:.2f}%")

        base_models.append(model)
        model_names.append(name)

    return base_models, model_names

## 7. TTA

In [ ]:
def mega_tta_predict(models, X_val_norm, n_tta=20):
    """강화된 TTA (정규화된 데이터 입력)"""
    print(f"\n🔮 Mega TTA (각 모델당 {n_tta}번 증강)")
    all_predictions = []

    for i, model in enumerate(tqdm(models, desc="TTA Models")):
        model_preds = []

        # 원본
        preds = model.predict(X_val_norm, batch_size=BATCH_SIZE*2, verbose=0)
        model_preds.append(preds)

        # 증강 버전들 (Numpy TTA)
        for _ in range(n_tta - 1):
            # TTA를 위해 배치 증강이 아닌 샘플별 증강 필요
            X_aug_list = []
            for sample in X_val_norm:
                aug_sample = mega_augment_batch(sample[np.newaxis, ...])[0]
                X_aug_list.append(aug_sample)
            X_aug = np.array(X_aug_list)

            preds_aug = model.predict(X_aug, batch_size=BATCH_SIZE*2, verbose=0)
            model_preds.append(preds_aug)

        avg_preds = np.mean(model_preds, axis=0)
        all_predictions.append(avg_preds)

    final_preds = np.mean(all_predictions, axis=0)
    return final_preds

## 8. 스태킹 앙상블

In [ ]:
def stacking_ensemble(models, X_train_norm, X_val_norm, y_train, y_val, NUM_CLASSES):
    """스태킹 앙상블 (메타 러너)"""
    print("\n🎯 스태킹 앙상블 학습")

    train_meta_features = []
    val_meta_features = []

    for i, model in enumerate(tqdm(models, desc="Stacking Features")):
        train_preds = model.predict(X_train_norm, batch_size=BATCH_SIZE*2, verbose=0)
        val_preds = model.predict(X_val_norm, batch_size=BATCH_SIZE*2, verbose=0)
        train_meta_features.append(train_preds)
        val_meta_features.append(val_preds)

    X_train_meta = np.concatenate(train_meta_features, axis=1)
    X_val_meta = np.concatenate(val_meta_features, axis=1)

    print(f"  메타 특징 shape: {X_train_meta.shape}")

    meta_model = tf.keras.Sequential([
        layers.Input(shape=(X_train_meta.shape[1],)),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(NUM_CLASSES, activation='softmax')
    ])

    meta_model.compile(
        optimizer=tf.keras.optimizers.Adam(0.001),
        loss='sparse_categorical_crossentropy', # 스태킹은 레이블 스무딩 없이
        metrics=['accuracy']
    )

    meta_model.fit(
        X_train_meta, y_train,
        validation_data=(X_val_meta, y_val),
        epochs=50,
        batch_size=BATCH_SIZE,
        callbacks=[
            callbacks.EarlyStopping(patience=10, restore_best_weights=True, verbose=0)
        ],
        verbose=0
    )

    stacked_preds = meta_model.predict(X_val_meta, verbose=0)
    return stacked_preds, meta_model

## 9. 메인 실행

In [ ]:
if __name__ == "__main__":

    # 1. 데이터 로드
    X_data, y_data, class_names = load_all_data(DATA_DIR, MAX_FRAMES, FEATURE_DIM)
    NUM_CLASSES = len(class_names)
    INPUT_SHAPE = (MAX_FRAMES, FEATURE_DIM)

    # 2. Train/Val 분할 (Data Leakage 방지!)
    print("\n✂️ Train/Val 분할...")
    X_train, X_val, y_train, y_val = train_test_split(
        X_data, y_data,
        test_size=VAL_SPLIT_SIZE,
        random_state=SEED,
        stratify=y_data
    )

    # 3. 정규화 (Train 기준으로 Val 정규화)
    print("🔄 데이터 정규화 (Standard Scaling)...")
    mean = X_train.mean(axis=(0, 1), keepdims=True)
    std = X_train.std(axis=(0, 1), keepdims=True) + 1e-6

    X_train_norm = (X_train - mean) / std
    X_val_norm = (X_val - mean) / std

    print(f"  Train: {len(X_train_norm)}개, Val: {len(X_val_norm)}개")

    # 4. tf.data.Dataset 생성
    # (주의: train_ds는 정규화되지 *않은* X_train을 사용 -> 증강 후 정규화)
    # (수정: 증강도 정규화된 데이터로 하는 것이 안정적일 수 있음. 여기서는 퀸의 원안대로)
    # (최종 수정: 정규화된 데이터로 증강하는 것이 맞음!)

    print("🚀 tf.data 파이프라인 생성 (실시간 증강 포함)...")
    train_ds = create_augmented_dataset(X_train_norm, y_train, BATCH_SIZE, mega_augment_batch)
    val_ds = tf.data.Dataset.from_tensor_slices((X_val_norm, y_val)).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

    # 5. 클래스 가중치 계산
    print("⚖️ 클래스 가중치 계산...")
    class_weights = compute_class_weight(
        class_weight="balanced",
        classes=np.unique(y_train),
        y=y_train
    )
    class_weights_dict = {i: w for i, w in enumerate(class_weights)}

    # 6. 하이브리드 앙상블 학습
    base_models, model_names = train_hybrid_ensemble(
        X_train_norm, X_val_norm, y_train, y_val,
        INPUT_SHAPE, NUM_CLASSES,
        train_ds, val_ds, class_weights_dict
    )

    # 7. 앙상블 성능 평가
    print("\n\n" + "="*60)
    print("📊 최종 성능 평가 (Validation Set)")
    print("="*60)

    # 방법 1: Averaging Ensemble
    avg_preds_raw = np.mean([model.predict(X_val_norm, verbose=0) for model in base_models], axis=0)
    avg_preds_labels = np.argmax(avg_preds_raw, axis=1)
    avg_acc = accuracy_score(y_val, avg_preds_labels)
    print(f"🏆 [Averaging] Ensemble Accuracy: {avg_acc*100:.2f}%")

    # 방법 2: Mega TTA
    tta_preds_raw = mega_tta_predict(base_models, X_val_norm, n_tta=10) # 시간상 10회
    tta_preds_labels = np.argmax(tta_preds_raw, axis=1)
    tta_acc = accuracy_score(y_val, tta_preds_labels)
    print(f"🚀 [Mega TTA] Ensemble Accuracy : {tta_acc*100:.2f}%")

    # 방법 3: Stacking Ensemble
    stacked_preds_raw, meta_model = stacking_ensemble(
        base_models, X_train_norm, X_val_norm, y_train, y_val, NUM_CLASSES
    )
    stacked_preds_labels = np.argmax(stacked_preds_raw, axis=1)
    stacking_acc = accuracy_score(y_val, stacked_preds_labels)
    print(f"👑 [Stacking] Ensemble Accuracy: {stacking_acc*100:.2f}%")

    print("\n" + "="*60)
    print("🎉 모든 학습 및 평가 완료.")
    print("="*60)

📦 데이터 로딩
📂 클래스: 67개


Loading: 100%|██████████| 67/67 [16:34<00:00, 14.85s/it]



✅ 총 샘플: 1229개

✂️ Train/Val 분할...
🔄 데이터 정규화 (Standard Scaling)...
  Train: 983개, Val: 246개
🚀 tf.data 파이프라인 생성 (실시간 증강 포함)...
⚖️ 클래스 가중치 계산...

🏗️ CNN_Attention_v1 학습 중... (LR: 0.001)
Epoch 1/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 26s 385ms/step - accuracy: 0.0122 - loss: 4.3078 - val_accuracy: 0.0203 - val_loss: 4.1999 - learning_rate: 0.0010
Epoch 2/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.0378 - loss: 4.2043 - val_accuracy: 0.0203 - val_loss: 4.1661 - learning_rate: 0.0010
Epoch 3/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - accuracy: 0.0232 - loss: 4.1469 - val_accuracy: 0.0528 - val_loss: 4.0065 - learning_rate: 0.0010
Epoch 4/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.0515 - loss: 4.0364 - val_accuracy: 0.0528 - val_loss: 3.9592 - learning_rate: 0.0010
Epoch 5/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.0474 - loss: 3.9255 - val_accuracy: 0.0976 - val_loss: 3.6801 - learning_rate: 0.0010
Epoch 6/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - 

  ✅ BiLSTM_Attention_v1 Val Acc: 75.61%

🏗️ BiLSTM_Attention_v2 학습 중... (LR: 0.0005)
Epoch 1/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 7s 62ms/step - accuracy: 0.0175 - loss: 4.5861 - val_accuracy: 0.0488 - val_loss: 4.1529 - learning_rate: 5.0000e-04
Epoch 2/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - accuracy: 0.0166 - loss: 4.1880 - val_accuracy: 0.0407 - val_loss: 4.0810 - learning_rate: 5.0000e-04
Epoch 3/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - accuracy: 0.0247 - loss: 4.1338 - val_accuracy: 0.0407 - val_loss: 4.0133 - learning_rate: 5.0000e-04
Epoch 4/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - accuracy: 0.0358 - loss: 4.1066 - val_accuracy: 0.0610 - val_loss: 3.9440 - learning_rate: 5.0000e-04
Epoch 5/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - accuracy: 0.0558 - loss: 3.9980 - val_accuracy: 0.1098 - val_loss: 3.7428 - learning_rate: 5.0000e-04
Epoch 6/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 40ms/step - accuracy: 0.0595 - loss: 3.9172 - val_accuracy: 0.1585 - val_loss: 3.5799 - learn

  ✅ BiLSTM_Attention_v2 Val Acc: 74.80%

🏗️ Transformer_v1 학습 중... (LR: 0.0008)
Epoch 1/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 34s 459ms/step - accuracy: 0.0164 - loss: 4.3150 - val_accuracy: 0.0691 - val_loss: 4.0631 - learning_rate: 8.0000e-04
Epoch 2/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.0378 - loss: 4.0440 - val_accuracy: 0.1016 - val_loss: 3.8010 - learning_rate: 8.0000e-04
Epoch 3/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.0787 - loss: 3.8982 - val_accuracy: 0.1992 - val_loss: 3.3971 - learning_rate: 8.0000e-04
Epoch 4/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.1925 - loss: 3.4742 - val_accuracy: 0.3089 - val_loss: 2.9983 - learning_rate: 8.0000e-04
Epoch 5/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 0.1868 - loss: 3.2049 - val_accuracy: 0.2846 - val_loss: 2.8200 - learning_rate: 8.0000e-04
Epoch 6/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 0.2800 - loss: 2.8974 - val_accuracy: 0.3902 - val_loss: 2.5554 - learning

TTA Models: 100%|██████████| 7/7 [00:14<00:00,  2.13s/it]


🚀 [Mega TTA] Ensemble Accuracy : 81.71%

🎯 스태킹 앙상블 학습


Stacking Features: 100%|██████████| 7/7 [00:05<00:00,  1.31it/s]


  메타 특징 shape: (983, 469)
👑 [Stacking] Ensemble Accuracy: 85.37%

🎉 모든 학습 및 평가 완료.
